# Import necessary libraries


In [10]:
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
import joblib

# Load the dataset

In [2]:
file_url = 'https://drive.google.com/uc?id=145FbzMbnxOZbSTV4oJQoKmaRffh86XeR'
output = 'diabetes.csv'
gdown.download(file_url, output, quiet=False)

df = pd.read_csv(output)
df.head()

Downloading...
From: https://drive.google.com/uc?id=145FbzMbnxOZbSTV4oJQoKmaRffh86XeR
To: /content/diabetes.csv
100%|██████████| 23.9k/23.9k [00:00<00:00, 13.5MB/s]


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
# Separate features and target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameter tuning for Logistic Regression


In [6]:
param_grid_lr = {'C': [0.01, 0.1, 1, 10, 100], 'solver': ['liblinear', 'lbfgs']}
grid_search_lr = GridSearchCV(LogisticRegression(random_state=42), param_grid_lr, cv=5, scoring='f1', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)
best_lr = grid_search_lr.best_estimator_
f1_lr = f1_score(y_test, best_lr.predict(X_test))
print(f"Best Logistic Regression F1 Score: {f1_lr}, with parameters: {grid_search_lr.best_params_}")

Best Logistic Regression F1 Score: 0.5874125874125874, with parameters: {'C': 1, 'solver': 'liblinear'}


# Hyperparameter tuning for Decision Tree



In [7]:
param_grid_dt = {'max_depth': [3, 5, 7, 10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
grid_search_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_dt, cv=5, scoring='f1', n_jobs=-1)
grid_search_dt.fit(X_train, y_train)
best_dt = grid_search_dt.best_estimator_
f1_dt = f1_score(y_test, best_dt.predict(X_test))
print(f"Best Decision Tree F1 Score: {f1_dt}, with parameters: {grid_search_dt.best_params_}")

Best Decision Tree F1 Score: 0.5531914893617021, with parameters: {'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 2}


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


# Hyperparameter tuning for Random Forest

In [8]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='f1', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
f1_rf = f1_score(y_test, best_rf.predict(X_test))
print(f"Best Random Forest F1 Score: {f1_rf}, with parameters: {grid_search_rf.best_params_}")

Best Random Forest F1 Score: 0.6164383561643836, with parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}


# Select the best model based on F1 score


In [12]:
best_model = None
best_f1 = 0
best_accuracy = 0

if f1_lr > f1_dt and f1_lr > f1_rf:
    best_model = best_lr
    best_f1 = f1_lr
    print("Best Model: Logistic Regression")
elif f1_dt > f1_lr and f1_dt > f1_rf:
    best_model = best_dt
    best_f1 = f1_dt
    print("Best Model: Decision Tree")
else:
    best_model = best_rf
    best_f1 = f1_rf
    print("Best Model: Random Forest")

Best Model: Random Forest


In [13]:
best_accuracy = accuracy_score(y_test, best_model.predict(X_test))
print(f"Best Model F1 Score: {best_f1}")
print(f"Best Model Accuracy: {best_accuracy}")

Best Model F1 Score: 0.6164383561643836
Best Model Accuracy: 0.7575757575757576


# Save the best model


In [14]:
model_filename = 'best_diabetes_model_tuned.sav'
joblib.dump(best_model, model_filename)
print(f"The best model has been saved as {model_filename}")

The best model has been saved as best_diabetes_model_tuned.sav
